In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `token2` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version:
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/12.3G [00:00<?, ?B/s]

Some weights of the model checkpoint at kakaobrain/kogpt were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.20.attn.bias'

In [ ]:
df_summary = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/비타민 14기/학기 NLP/new_output_with_summary_200.csv')

## kogpt 훈련 코드
메모리 부족으로 train 진행하지 않음

In [ ]:
class ConversationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self._data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        # Retrieve the sample
        item = self._data.iloc[idx]  # Access the row by index
        prompt = f"주제: {item['subject']} 요약: {item['summary']} 대화: "
        target = item['text']

        # Tokenize the input (prompt) and target (text)
        inputs = self.tokenizer(
            prompt,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        labels = self.tokenizer(
            target,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        labels = labels['input_ids'].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": inputs['input_ids'].squeeze(),
            "attention_mask": inputs['attention_mask'].squeeze(),
            "labels": labels
        }


In [ ]:
from torch.utils.data import Dataset
train_dataset = ConversationDataset(df_summary[:180], tokenizer)
valid_dataset = ConversationDataset(df_summary[180:], tokenizer)

eval로 텍스트 생성

In [ ]:
# 따로 프롬프트 없이 요약 텍스트만 입력하는 것이 생성 성능은 가장 우수함
_ = model.eval()
prompt_vanila = 'Q:' + df_summary['summary'][85] + 'A:'

with torch.no_grad():
  tokens = tokenizer.encode(prompt_vanila, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=80, eos_token_id=tokenizer.eos_token_id,
                              top_p=0.9, top_k=50, repetition_penalty=1.2)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

Q:귀멸의 칼날 작가가 은퇴한 것에 대해 이야기하고 있다.A:은퇴하기엔 작품이 너무 많았다.Q:이번주 일본 서점에서 인기순위 1위를 기록했는데?A:저도 알고있었다. Q:일본 독자들은 왜 귀살대를 좋아하는가?A:일단 캐릭터성이 독특하다.


In [ ]:
prompt_new = 'Q: 코딩 과제의 난이도가 어렵다는 내용에 대해 이야기하고 있다. A: '

with torch.no_grad():
  tokens = tokenizer.encode(prompt_new, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=80, eos_token_id=tokenizer.eos_token_id,
                              top_p=0.9, top_k=50, repetition_penalty=1.2)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

Q: 코딩 과제의 난이도가 어렵다는 내용에 대해 이야기하고 있다. A:  어려운 건 사실이다. 특히나 우리 학교에서는 소프트웨어를 배울 때 파이썬을 배우는데, 정말 어려워서 좌절하기도 한다. 그래서 그런지 학생들 중에서도 '코딩' 과목은 내후년에도 안 듣겠다는 사람이 많아졌고, 이런 현상 때문에 다른 교수님들이
